In [ ]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
import gc
import utils


TEST_SET_THRESHOLD = 10
TEST_SET_HOLDOUT = 0.2
BEST_ALFA = 0.8
BEST_BETA = 0.9
BEST_BATCH = 10
BEST_K = 50

In [ ]:
tracks = pd.read_csv('../input/tracks.csv')
train = pd.read_csv('../input/train.csv')
target = pd.read_csv('../input/target_playlists.csv')

In [ ]:
icm_csr = utils.build_icm_csr(tracks)
urm_csr = utils.build_urm_csr(train)

In [ ]:
from Notebooks_utils.data_splitter import train_test_holdout

URM_train, URM_test = train_test_holdout(urm_csr, train_perc = 0.8)
URM_train, URM_validation = train_test_holdout(URM_train, train_perc = 0.9)

In [ ]:
class EnsembleRecommender(object):
    
    def get_URM_train(self):
        return self.URM_csr.copy()
    
    def fit(self,slim_recommender, URM_csr, ICM_csr, alfa, beta):
        self.beta = beta
        self.slim_recommender = slim_recommender
        transformer = TfidfTransformer()
        transformer.fit(URM_csr)
        tf_idf_csr = transformer.transform(URM_csr)

        IRM = sparse.csr_matrix(tf_idf_csr.transpose())
        
        csr_similarities = sparse.csr_matrix(cosine_similarity(IRM, dense_output=False))
        

        transformer.fit(ICM_csr)
        tf_idf_icm = transformer.transform(ICM_csr)
        icm_similarities = sparse.csr_matrix(cosine_similarity(tf_idf_icm, dense_output=False))
        
        print("COMPUTING ENSEMBLE SIMILARITIES")
        self.item_similarities = alfa*csr_similarities + (1-alfa)*icm_similarities        
        self.URM_csr = URM_csr
        
    
    def recommend(self, user_id, cutoff=10, remove_seen_flag=True):
        
        
        user = self.URM_csr.getrow(user_id)
        itemPopularity = user.dot(self.item_similarities)
        slimPopularity = self.slim_recommender.compute_item_score(user_id)
        item_popularity = itemPopularity*self.beta + slimPopularity*(1-self.beta)
        popularItems = np.argsort(np.array(item_popularity)[0])
        popularItems = np.flip(popularItems, axis = 0)

        if remove_seen_flag:
            unseen_items_mask = np.in1d(popularItems, self.URM_csr[user_id].indices,
                                        assume_unique=True, invert = True)

            unseen_items = popularItems[unseen_items_mask]
            
            recommended_items = unseen_items[0:cutoff]

        else:
            recommended_items = popularItems[0:cutoff]
            
        #recommended_items = " ".join(str(i) for i in recommended_items)
        return recommended_items
    
    

In [ ]:
recommender = SLIM_BPR_Cython(URM_train,recompile_cython=False,positive_threshold=1) 

In [ ]:
recommender.fit(epochs=150, batch_size=5,sgd_mode='adam',learning_rate=1e-4,topK=80)

In [ ]:
ensemble_recommender = EnsembleRecommender()
ensemble_recommender.fit(recommender,urm_csr,icm_csr,alfa = 0.8,beta = 0.9)

In [ ]:
target = target.get_values()[:,0]

In [ ]:
n = 500
  
# using list comprehension 
divided_target = [target[i * n:(i + 1) * n] for i in range((len(target) + n - 1) // n )]  

In [ ]:
result = []
for items in target:
    partial_recomendations = ensemble_recommender.recommend(items,cutoff=10)
    result.append(partial_recomendations)

In [ ]:
result = np.array(result)

In [ ]:
result = result.reshape(-1,10)

In [ ]:
i = 0
results = []
for item in result:
    recommended_items = " ".join(str(i) for i in item)
    temp = [target[i],recommended_items]
    results.append(temp)
    i += 1

In [ ]:
rec = pd.DataFrame(results)
rec.to_csv("submission_ensemble.csv", index = False, header = ["playlist_id", "track_ids"])